実装にあたってこちらの記事を参考にさせていただきました  
[Zenn 歩行時の加速度,角速度データから歩行軌跡を描いてみた - Beyond Toyama](https://zenn.dev/happy663/articles/1a0abc285b3a33)

# 事前準備

## ライブラリのインポート

In [131]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [188]:
import time
import requests
import random
import csv
import subprocess
import os

## ユーザ用意

### 必要なimport

In [166]:
import uuid
import string

### 関数を定義

In [167]:
def generate_random_hex():
    return uuid.uuid4().hex  # 32文字の16進数を生成

def generate_random_name(length=8):
    return ''.join(random.choices(string.ascii_lowercase, k=length))

def generate_sql_entry(id):
    timestamp = "2024-04-20 21:59:08.932"
    random_hex = generate_random_hex()
    random_name = generate_random_name()
    return f"({id},'{timestamp}','{timestamp}',NULL,'{random_hex}','{random_name}','',1,1,2,NULL),"

### 実行

In [186]:
for i in range(100):
  users_num = 100 * (i+1)
  # SQLファイルの初期部分
  sql_head = f"""DROP TABLE IF EXISTS users;
  CREATE TABLE users (
    id bigint unsigned NOT NULL AUTO_INCREMENT,
    created_at datetime(3) DEFAULT NULL,
    updated_at datetime(3) DEFAULT NULL,
    deleted_at datetime(3) DEFAULT NULL,
    uuid longtext,
    name longtext,
    email longtext,
    role bigint DEFAULT NULL,
    beacon_id bigint DEFAULT NULL,
    community_id bigint DEFAULT NULL,
    private_key longtext,
    PRIMARY KEY (id),
    KEY idx_users_deleted_at (deleted_at)
  ) ENGINE=InnoDB AUTO_INCREMENT={users_num} DEFAULT CHARSET=utf8mb3;

  LOCK TABLES users WRITE;
  INSERT INTO users VALUES 
  """

  sql_footer = f"""
  ({users_num},'2024-04-20 21:59:08.932','2024-11-26 15:57:08.286',NULL,'8ebc21144abdba0db7c6ff0a00200016','togawa','toge7113@gmail.com',2,1,2,NULL);
  UNLOCK TABLES;
  """

  # 例として5つのデータを生成
  entries = [generate_sql_entry(i) for i in range(1, users_num)]

  # SQL出力
  sql_body = ""
  sql_body += sql_body + "\n".join(entries)
  sql_content = sql_head + sql_body + sql_footer
  # print(sql_content)

  # ファイルに書き込み
  with open(f"test_users/uuid_method/users{users_num}.dump", "w", encoding="utf-8") as f:
      f.write(sql_content)

  print(f"users{users_num}.dumpを生成しました。")

users100.dumpを生成しました。
users200.dumpを生成しました。
users300.dumpを生成しました。
users400.dumpを生成しました。
users500.dumpを生成しました。
users600.dumpを生成しました。
users700.dumpを生成しました。
users800.dumpを生成しました。
users900.dumpを生成しました。
users1000.dumpを生成しました。
users1100.dumpを生成しました。
users1200.dumpを生成しました。
users1300.dumpを生成しました。
users1400.dumpを生成しました。
users1500.dumpを生成しました。
users1600.dumpを生成しました。
users1700.dumpを生成しました。
users1800.dumpを生成しました。
users1900.dumpを生成しました。
users2000.dumpを生成しました。
users2100.dumpを生成しました。
users2200.dumpを生成しました。
users2300.dumpを生成しました。
users2400.dumpを生成しました。
users2500.dumpを生成しました。
users2600.dumpを生成しました。
users2700.dumpを生成しました。
users2800.dumpを生成しました。
users2900.dumpを生成しました。
users3000.dumpを生成しました。
users3100.dumpを生成しました。
users3200.dumpを生成しました。
users3300.dumpを生成しました。
users3400.dumpを生成しました。
users3500.dumpを生成しました。
users3600.dumpを生成しました。
users3700.dumpを生成しました。
users3800.dumpを生成しました。
users3900.dumpを生成しました。
users4000.dumpを生成しました。
users4100.dumpを生成しました。
users4200.dumpを生成しました。
users4300.dumpを生成しました。
users4400.dumpを生成しまし

# API実行時間計測

In [138]:
url = "http://localhost:8082/api/v1/stayers"
baseURL = "http://localhost:8082"

# ヘッダー
headers = {
    "Content-Type": "application/json"
}

## ユーザ登録用関数の定義（APIを用いて）

従来手法のユーザの登録

In [195]:
def restoreUsersTable(numUsers):
  # 変数の値を指定
  db_container_name = "vol_mysql"
  mysql_password = "root"
  db_name = "app"
  dump_file = f"test_users/uuid_method/users{numUsers}.dump"

  # 実行するコマンド
  command = f"docker exec -i {db_container_name} mysql -uroot -p{mysql_password} {db_name} < {dump_file}"
  # シェルスクリプトを実行
  result = subprocess.run(command, shell=True, capture_output=True, text=True, env=os.environ)
  print(result)
  print("ユーザ登録完了")

In [197]:
# シェルスクリプトを実行
restoreUsersTable(200)

CompletedProcess(args='docker exec -i vol_mysql mysql -uroot -proot app < test_users/uuid_method/users200.dump', returncode=0, stdout='', stderr='mysql: [Warning] Using a password on the command line interface can be insecure.\n')
ユーザ登録完了


## リクエストを定義

### ボディデータを設定する関数を定義

In [163]:
# 従来手法の未登録のユーザをリクエストに含める
def getRequestBasicNonRegisterdUser(numUsers):
    data = {
        "beacons": [],
        "roomId": 1
    }
    for i in range(numUsers):
        hexString = ''.join(random.choices('0123456789abcdef', k=32))
        # ボディデータ
        data["beacons"].append({
            "uuid": hexString,
            "rssi": -49
        })
    print(data)
    return data

# 従来手法の登録済みのユーザをリクエストに含める
def getRequestBasicRegisterdUser(numUsers):
    responseUsers = requests.get(baseURL+"/api/v1/users/extended")
    users=responseUsers.json()
    # ボディデータ
    data = {
        "beacons": [],
        "roomId": 1
    }
    for i in range(numUsers):
        randomUser = random.choice(users)  # ランダムに 1 人選択
        # ボディデータ
        data["beacons"].append({
            "uuid": randomUser["uuid"],
            "rssi": -49
        })
    print(data)
    return data

## 計測

### 従来手法での計測

In [199]:
FILE_PATH = "out/basic02.csv"
with open(FILE_PATH, 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['users', 'time', 'user_state', 'status'])

print("start bench")

for i in range(4):
  print(str(i+1) + "/100回目")
  # 100人ランダムな従来ユーザを登録する（PythonでWebAPIを用いて）
  restoreUsersTable(100*(i+1))

  # ランダムなユーザ(登録済み)を20人POSTで投げる
  req_json = getRequestBasicRegisterdUser(20)
  start = time.perf_counter()
  response = requests.post(url, json=req_json, headers=headers)
  end = time.perf_counter()
  with open(FILE_PATH, 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([100*(i+1), end - start, 'Registerd', response.json()])  # 1行追加

  # ランダムなユーザ(未登録)を20人POSTで投げる
  req_json = getRequestBasicNonRegisterdUser(20)
  start = time.perf_counter()
  response = requests.post(url, json=req_json, headers=headers)
  end = time.perf_counter()
  with open(FILE_PATH, 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([100*(i+1), end - start, 'NonRegisterd', response.json()])  # 1行追加


# start = time.perf_counter()

# response = requests.post(url, json=getRequestBasicNonRegisterdUser(10), headers=headers)

# end = time.perf_counter()
# print(response.json())
# print("end bench")
# print("実行時間: " + str(end - start) + " sec")

start bench
1/100回目
CompletedProcess(args='docker exec -i vol_mysql mysql -uroot -proot app < test_users/uuid_method/users100.dump', returncode=0, stdout='', stderr='mysql: [Warning] Using a password on the command line interface can be insecure.\n')
ユーザ登録完了
{'beacons': [{'uuid': '7e50f07fffff4df29d5364ce1c11c5c2', 'rssi': -49}, {'uuid': '5446f2fa602f41b6a94524ec38c3f9d2', 'rssi': -49}, {'uuid': 'd78e8f2c67aa4f23a4ab299b5e289907', 'rssi': -49}, {'uuid': '17f8d7ed9634474fbfc7018e5c9c9678', 'rssi': -49}, {'uuid': '5913f4b995494ac9bcd5efad487a9c03', 'rssi': -49}, {'uuid': 'cdd0b294f0ec4a5ba9d732358e606473', 'rssi': -49}, {'uuid': '1f811e5f4c654315bb7a8dc1d23d7794', 'rssi': -49}, {'uuid': 'ca03623f2a1949ca80bc9dfb9cc92a04', 'rssi': -49}, {'uuid': 'f45d2be099214981b11e4d3fcdb1d5e7', 'rssi': -49}, {'uuid': '1587cb07307f42c4bb380effc502e3fd', 'rssi': -49}, {'uuid': '13921716fb7b48cf9f936c755b0a9c65', 'rssi': -49}, {'uuid': '537f89177fc643ecbabd66c1a34b42a8', 'rssi': -49}, {'uuid': 'a69513e121

### 滞在情報をリセット

In [96]:
# ボディデータ
data = {
    "beacons": [
    ],
    "roomId": 1
}
response = requests.post(url, json=data, headers=headers)